In [ ]:
from google.colab import drive

drive.mount('./drive')

Mounted at ./drive


In [ ]:
from shutil import copytree
import os
import time
import datetime
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
import random
from IPython import display
from tqdm import tqdm
import PIL.Image

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,optimizers,losses
from tensorflow.python.keras.utils import conv_utils
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.python.keras import backend as K
from keras.layers import Input, Activation, Add, UpSampling2D, BatchNormalization, Conv2D, Dropout, Conv2DTranspose, Lambda, Flatten, Dense, Layer, InputSpec
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Model
from tensorflow.keras.utils import plot_model
from IPython.display import Image
from tensorflow.keras.applications.vgg16 import VGG16
from keras.callbacks import TensorBoard

import cv2
import PIL
import json, os
import sys

import glob
import itertools

In [ ]:
copytree('drive/MyDrive/Colab Notebooks/deblurring_module', '/content/deblurring_module')
!mkdir GOPRO_Large

!unzip -qq "/content/drive/MyDrive/Colab Notebooks/GOPRO_Large.zip" -d "/content/GOPRO_Large"

In [ ]:
# 데이터 불러오기
from deblurring_module.data_load import reorganize_gopro_files
reorganize_gopro_files('GOPRO_Large/', 'images')

dir: 100%|██████████| 2/2 [00:48<00:00, 24.24s/it]


In [ ]:
from deblurring_module.images import load_images
data = load_images('./images/train', -1)
y_train, x_train = data['B'], data['A']

In [ ]:
x_train = (x_train - 127.5) / 127.5
y_train = (y_train - 127.5) / 127.5

In [ ]:
def res_block(input, filters, kernel_size=(3,3), strides=(1,1), use_dropout=False):

    x = Conv2D(filters=filters, kernel_size=kernel_size, strides=strides,padding='same')(input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    if use_dropout:
        x = Dropout(0.5)(x)

    x = Conv2D(filters=filters,kernel_size=kernel_size,strides=strides,padding='same')(x)
    x = BatchNormalization()(x)

    #  Two convolution layers are connected between input and output 
    merged = Add()([input, x])
    return merged

In [ ]:
def spatial_reflection_2d_padding(x, padding=((1, 1), (1, 1)), data_format=None):
    assert len(padding) == 2
    assert len(padding[0]) == 2
    assert len(padding[1]) == 2
    if data_format is None:
        data_format = image_data_format()
    if data_format not in {'channels_first', 'channels_last'}:
        raise ValueError('Unknown data_format ' + str(data_format))

    if data_format == 'channels_first':
        pattern = [[0, 0],
                   [0, 0],
                   list(padding[0]),
                   list(padding[1])]
    else:
        pattern = [[0, 0],
                   list(padding[0]), list(padding[1]),
                   [0, 0]]
    return tf.pad(x, pattern, "REFLECT")

In [ ]:
class ReflectionPadding2D(Layer):
    def __init__(self,
                 padding=(1, 1),
                 data_format=None,
                 **kwargs):
        super(ReflectionPadding2D, self).__init__(**kwargs)
        self.data_format = conv_utils.normalize_data_format(data_format)
        if isinstance(padding, int):
            self.padding = ((padding, padding), (padding, padding))
        elif hasattr(padding, '__len__'):
            if len(padding) != 2:
                raise ValueError('`padding` should have two elements. '
                                 'Found: ' + str(padding))
            height_padding = conv_utils.normalize_tuple(padding[0], 2,
                                                        '1st entry of padding')
            width_padding = conv_utils.normalize_tuple(padding[1], 2,
                                                       '2nd entry of padding')
            self.padding = (height_padding, width_padding)
        else:
            raise ValueError('`padding` should be either an int, '
                             'a tuple of 2 ints '
                             '(symmetric_height_pad, symmetric_width_pad), '
                             'or a tuple of 2 tuples of 2 ints '
                             '((top_pad, bottom_pad), (left_pad, right_pad)). '
                             'Found: ' + str(padding))
        self.input_spec = InputSpec(ndim=4)

    def compute_output_shape(self, input_shape):
        if self.data_format == 'channels_first':
            if input_shape[2] is not None:
                rows = input_shape[2] + self.padding[0][0] + self.padding[0][1]
            else:
                rows = None
            if input_shape[3] is not None:
                cols = input_shape[3] + self.padding[1][0] + self.padding[1][1]
            else:
                cols = None
            return (input_shape[0],
                    input_shape[1],
                    rows,
                    cols)
        elif self.data_format == 'channels_last':
            if input_shape[1] is not None:
                rows = input_shape[1] + self.padding[0][0] + self.padding[0][1]
            else:
                rows = None
            if input_shape[2] is not None:
                cols = input_shape[2] + self.padding[1][0] + self.padding[1][1]
            else:
                cols = None
            return (input_shape[0],
                    rows,
                    cols,
                    input_shape[3])

    def call(self, inputs):
        return spatial_reflection_2d_padding(inputs,
                                             padding=self.padding,
                                             data_format=self.data_format)

    def get_config(self):
        config = {'padding': self.padding,
                  'data_format': self.data_format}
        base_config = super(ReflectionPadding2D, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [ ]:
image_shape = (256, 256, 3)
ngf = 64
input_nc = 3
batch_size = 16
output_nc = 3

input_shape_generator = (256, 256, input_nc)
n_block_gen = 9

def generator_model():
    """ Build a generation model """
    # Current version : ResNet block
    inputs = Input(shape=(256,256,3))

    x = ReflectionPadding2D((3, 3))(inputs)
    x = Conv2D(filters=ngf, kernel_size=(7, 7), padding='valid')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Increase filter number
    n_downsampling = 2
    for i in range(n_downsampling):
        mult = 2 ** i
        x = Conv2D(filters=ngf * mult * 2, kernel_size=(3, 3), strides=2, padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

    # 9 ResNet blocks
    mult = 2 ** n_downsampling
    for i in range(n_block_gen):
        x = res_block(x, ngf * mult, use_dropout=True)

    #  Reduce the convolution kernel to 3 individual  (RGB)
    for i in range(n_downsampling):
        mult = 2 ** (n_downsampling - i)
        x = UpSampling2D()(x) 
        x = Conv2D(filters=int(ngf * mult / 2), kernel_size=(3, 3), padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

    x = ReflectionPadding2D((3, 3))(x)
    x = Conv2D(filters=output_nc, kernel_size=(7, 7), padding='valid')(x)
    x = Activation('tanh')(x)

    # Add direct connection from input to output and recenter to [-1, 1]
    outputs = Add()([x, inputs])
    outputs = Lambda(lambda z: z / 2)(outputs)

    model = Model(inputs=inputs, outputs=outputs, name='Generator')
    return model

In [ ]:
ndf = 64

def discriminator_model():
    """ Construct discrimination model ."""
    n_layers, use_sigmoid = 3, False
    inputs = Input(shape=(256, 256, 3))

    x = Conv2D(filters=ndf, kernel_size=(4, 4), strides=2, padding='same')(inputs)
    x = LeakyReLU(0.2)(x)

    nf_mult, nf_mult_prev = 1, 1
    for n in range(n_layers):
        nf_mult_prev, nf_mult = nf_mult, min(2 ** n, 8)
        x = Conv2D(filters=ndf * nf_mult, kernel_size=(4, 4), strides=2, padding='same')(x)
        x = BatchNormalization()(x)
        x = LeakyReLU(0.2)(x)

    nf_mult_prev, nf_mult = nf_mult, min(2 ** n_layers, 8)
    x = Conv2D(filters=ndf * nf_mult, kernel_size=(4, 4), strides=1, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(0.2)(x)

    x = Conv2D(filters=1, kernel_size=(4, 4), strides=1, padding='same')(x)
    if use_sigmoid:
        x = layers.Activation('sigmoid')(x)

    x = Flatten()(x)
    x = Dense(1024, activation='tanh')(x)
    x = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=inputs, outputs=x, name='Discriminator')
    return model

In [ ]:
def generator_containing_discriminator_multiple_outputs(generator, discriminator):
    inputs = Input(shape=image_shape)
    generated_image = generator(inputs)
    outputs = discriminator(generated_image)
    model = Model(inputs=inputs, outputs=[generated_image, outputs])
    return model

In [ ]:
generator = generator_model()
#pretrained_path = '/content/drive/MyDrive/photo_results/725/generator_59_25.01026725769043.h5'
#generator.load_weights(pretrained_path)

In [ ]:
discriminator = discriminator_model()
# dis_pretrained_path = '/content/drive/MyDrive/photo_results/725/discriminator_59.h5'
# discriminator.load_weights(dis_pretrained_path)

In [ ]:
# cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

# def discriminator_loss(real_output, fake_output):
#     real_loss = cross_entropy(tf.ones_like(real_output), real_output)
#     fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
#     total_loss = real_loss + fake_loss
#     return total_loss

# def generator_loss(fake_output):
#     return cross_entropy(tf.ones_like(fake_output), fake_output)

def perceptual_loss(y_true, y_pred):
    vgg = VGG16(include_top=False, weights='imagenet', input_shape=image_shape)
    loss_model = Model(inputs=vgg.input, outputs=vgg.get_layer('block3_conv3').output)
    loss_model.trainable = False
    return K.mean(K.square(loss_model(y_true) - loss_model(y_pred)))

def wasserstein_loss(y_true, y_pred):
    return K.mean(y_true*y_pred)

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(learning_rate=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

In [ ]:
# def train(train, label, epochs):
#     for epoch in tqdm(range(epochs)):
#         start = time.time()
#         permutated_indexes = np.random.permutation(train.shape[0])

#         len_index = int(train.shape[0]/batch_size)
        
#         for index in range(len_index):
#             with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
#                 batch_indexes = permutated_indexes[index*batch_size:(index+1)*batch_size]
#                 image_blur_batch = train[batch_indexes]
#                 image_sharp_batch = label[batch_indexes]

#                 generated_images = generator(image_blur_batch, training=True)

#                 real_output = discriminator(image_sharp_batch, training=True)
#                 fake_output = discriminator(generated_images, training=True)

#                 gen_loss = generator_loss(fake_output)
#                 disc_loss = discriminator_loss(real_output, fake_output)
              
#             gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
#             gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

#             generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
#             discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

#         display.clear_output(wait=True)

#         if (epoch + 1) % 15 == 0:
#             save_all_weights(discriminator, generator, epoch, gen_loss)
#         print(f'Time for epoch {epoch+1} is {time.time()-start} sec')
#     display.clear_output(wait=True)

In [ ]:
def train_multiple_outputs(batch_size, epoch_num, critic_updates=5):
    g = generator_model()
    d = discriminator_model()
    d_on_g = generator_containing_discriminator_multiple_outputs(g, d)

    d_opt = tf.keras.optimizers.Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    d_on_g_opt = tf.keras.optimizers.Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

    d.trainable = True
    d.compile(optimizer=d_opt, loss=wasserstein_loss)
    d.trainable = False
    loss = [perceptual_loss, wasserstein_loss]
    loss_weights = [100, 1]
    d_on_g.compile(optimizer=d_on_g_opt, loss=loss, loss_weights=loss_weights)
    d.trainable = True

    output_true_batch, output_false_batch = np.ones((batch_size, 1)), -np.ones((batch_size, 1))

    # log_path = './logs'
    # tensorboard_callback = TensorBoard(log_path)

    for epoch in tqdm(range(epoch_num)):
        permutated_indexes = np.random.permutation(x_train.shape[0])

        d_losses = []
        d_on_g_losses = []
        for index in range(int(x_train.shape[0] / batch_size)):
            batch_indexes = permutated_indexes[index*batch_size:(index+1)*batch_size]
            image_blur_batch = x_train[batch_indexes]
            image_full_batch = y_train[batch_indexes]

            generated_images = g.predict(x=image_blur_batch, batch_size=batch_size)

            for _ in range(critic_updates):
                d_loss_real = d.train_on_batch(image_full_batch, output_true_batch)
                d_loss_fake = d.train_on_batch(generated_images, output_false_batch)
                d_loss = 0.5 * np.add(d_loss_fake, d_loss_real)
                d_losses.append(d_loss)

            d.trainable = False

            d_on_g_loss = d_on_g.train_on_batch(image_blur_batch, [image_full_batch, output_true_batch])
            d_on_g_losses.append(d_on_g_loss)

            d.trainable = True

        # write_log(tensorboard_callback, ['g_loss', 'd_on_g_loss'], [np.mean(d_losses), np.mean(d_on_g_losses)], epoch_num)
        print(np.mean(d_losses), np.mean(d_on_g_losses))
        # with open('log.txt', 'a+') as f:
        #     f.write('{} - {} - {}\n'.format(epoch, np.mean(d_losses), np.mean(d_on_g_losses)))

        save_all_weights(d, g, epoch, int(np.mean(d_on_g_losses)))

In [ ]:
BASE_DIR = '/content/drive/MyDrive/photo_results/'

def save_all_weights(d, g, epoch_number, current_loss):
    now = datetime.datetime.now()
    save_dir = os.path.join(BASE_DIR, '{}{}'.format(now.month, now.day))
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    g.save_weights(os.path.join(save_dir, 'generator_{}_{}.h5'.format(epoch_number, current_loss)), True)
    d.save_weights(os.path.join(save_dir, 'discriminator_{}.h5'.format(epoch_number)), True)

In [ ]:
# %%time
train_multiple_outputs(16, 100)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
  0%|          | 0/100 [00:00<?, ?it/s]

58900480/58889256 [==============================] - 1s 0us/step


  0%|          | 0/100 [00:18<?, ?it/s]


ValueError: ignored

In [ ]:
# %%time
# train(x_train, y_train, 300)

 18%|█▊        | 53/300 [1:54:44<8:53:34, 129.61s/it]

Time for epoch 53 is 129.69302225112915 sec


In [ ]:
Richter_path = '/content/drive/MyDrive/Colab Notebooks/Richter'

In [ ]:
RESHAPE = (256, 256)

def load_image(path):
    img = PIL.Image.open(path)
    return img

def preprocess_image(cv_img):
    cv_img = cv_img.resize(RESHAPE)
    img = np.array(cv_img)
    img = (img - 127.5) / 127.5
    return img

def deprocess_image(img):
    img = img * 127.5 + 127.5
    return img.astype('uint8')

def save_image(np_arr, path):
    img = np_arr * 127.5 + 127.5
    im = PIL.Image.fromarray(img)
    im.save(path)

In [ ]:
output_path = '/content/drive/MyDrive/photo_results/'
pretrained_path = '/content/drive/MyDrive/photo_results/725/generator.h5'

def deblur(weight_path, input_dir, output_dir):
    generator = generator_model()
    generator.load_weights(weight_path)
    for image_name in os.listdir(input_dir):
        image = np.array([preprocess_image(load_image(os.path.join(input_dir, image_name)))])
        x_test = image
        generated_images = generator.predict(x=x_test)
        generated = np.array([deprocess_image(img) for img in generated_images])
        x_test = deprocess_image(x_test)
        for i in range(generated_images.shape[0]):
            x = x_test[i, :, :, :]
            img = generated[i, :, :, :]
            output = np.concatenate((x, img), axis=1)
            im = PIL.Image.fromarray(output.astype(np.uint8))
            im.save(os.path.join(output_dir, image_name))

In [ ]:
generator.summary()

In [ ]:
deblur(pretrained_path, Richter_path, output_path)